# **Exercise 3: Pricing Analysis**

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.optimize import minimize_scalar

## Part I: Pricing Analysis
---
- **Optimal pricing for weekend vs. weekday**
- **Dataset:** demand_data_full

Consider the objective where you would like to know how we should adjust prices between weekday and weekend to Maximize Profit. That is, given the information below on demand d(p) and cost below, as well as the data on price, demand, and weekend/weekday, the objective is to
obtain two optimal prices and the corresponding profit for both cases.  

Additional info:
- Linear Price-response Curve: d(p) = 𝛽0 + 𝛽1p + 𝛽2𝑤
- Assume unit cost of eggs is $4: c = 4
- Unknown decision variable to optimize: p

In [ ]:
demand_data = pd.read_excel('/content/demand_data_full.xlsx')
demand_data.head()

,price,demand,weekend
0,10.5,14,0
1,15.0,12,0
2,6.0,16,0
3,12.0,32,1
4,21.0,8,0


In [ ]:
demand_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   price    260 non-null    float64
 1   demand   260 non-null    int64  
 2   weekend  260 non-null    int64  
dtypes: float64(1), int64(2)
memory usage: 6.2 KB


In [ ]:
demand_data.groupby('weekend')['demand'].mean()

weekend
0    12.248705
1    28.298507
Name: demand, dtype: float64

### 1.1 Regression

- Estimate the demand d(p) by regressing on price and weekend.
- Interpret the regression results. Provide an interpretation in your own words!

In [ ]:
model_ols = smf.ols(formula='demand ~ price + weekend', data=demand_data).fit()
print(model_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 demand   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                     2092.
Date:                Sun, 07 Apr 2024   Prob (F-statistic):          9.57e-160
Time:                        17:16:31   Log-Likelihood:                -551.52
No. Observations:                 260   AIC:                             1109.
Df Residuals:                     257   BIC:                             1120.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     22.7331      0.352     64.662      0.0

### 1.2 Weekday

Calculate the optimal weekday price p* (when w = 0) and its corresponding profit
- Clearly state your findings, do not simply show output of running code.
- Determine whether the optimal weekday price is within the credible range and provide response in your own words!

In [ ]:
c = 4
intercept = model_ols.params[0]
price_coef = model_ols.params[1]
weekend_coef = model_ols.params[2]

# Write objective: profit
# Add -1 to the minimizer if you want to maximize
# D(p)(p-c)
pi_weekday = lambda p: -(model_ols.params[0] + model_ols.params[1]*p) * (p-c)

# Calculating optimal price p* for weekday (w=0)
optimal_weekday = minimize_scalar(pi_weekday, method='brent')
optimal_price_weekday = optimal_weekday.x

print("Weekday Optimal Price: $", np.round(optimal_price_weekday, 2))

Weekday Optimal Price: $ 16.47


In [ ]:
min_price = demand_data.price.min()
max_price = demand_data.price.max()

print(f'(Min) {min_price}', ' < ', round(optimal_price_weekday, 2), ' < ', f'{max_price} (Max)')
min_price < optimal_price_weekday < max_price

(Min) 4.5  <  16.47  <  22.5 (Max)


True

In [ ]:
optimal_profit_weekday = -optimal_weekday.fun

print("Corresponding profit for Optimal Weekday Price is: $", np.round(optimal_profit_weekday, 2))

Corresponding profit for Optimal Weekday Price is: $ 122.2


### 1.3 Weekend

Calculate the optimal weekend price p* (when w = 1) and its corresponding profit.
- Clearly state your findings, do not simply show output of running code.
- Determine whether the optimal week.

In [ ]:
# Write objective: profit
# Add -1 to the minimizer if you want to maximize
# D(p)(p-c)
pi_weekend = lambda p: -(model_ols.params[0] + model_ols.params[1]*p + model_ols.params[2]*1) * (p-c)

# Calculating optimal price p* for weekday (w=0)
optimal_weekend = minimize_scalar(pi_weekend, method='brent')
optimal_price_weekend = optimal_weekend.x

print("Weekend Optimal Price: $", np.round(optimal_price_weekend, 2))

Weekend Optimal Price: $ 26.67


In [ ]:
print(f'(Min) {min_price}', ' < ', round(optimal_price_weekend, 2), ' < ', f'{max_price} (Max)')
min_price < optimal_price_weekend < max_price

(Min) 4.5  <  26.67  <  22.5 (Max)


False

In [ ]:
optimal_profit_weekend = -optimal_weekend.fun

print("Corresponding profit for Optimal Weekend Price is: $", np.round(optimal_profit_weekend, 2))

Corresponding profit for Optimal Weekend Price is: $ 403.72


## Part II: Pricing Analysis
---
- **New vs Refurbished Product**  
- **Dataset:** refurb_data

Consider the objective where you would like to know how we should adjust our refurbished prices depending on the range of new price offerings: 300, 350, 400 (verify all possible values by looking at the data yourself). The objective is to set optimal refurbished prices for all ranges of new prices and the corresponding profit for each case.

As a start, you need to get a sense of demand d(p) for the refurbished price offerings. This can be achieved by tallying up (counting) all of the instances in the data when the consumer chose the refurbished product, on a day-to-day basis. And this daily demand representing the cumulative ‘refurb’ outcomes, could be used as your dependent variable for a d(p) regression.

Additional info:
- Linear Price-response Curve: d(p) = 𝛽0 + 𝛽1𝑟𝑒𝑓𝑢𝑟𝑏_𝑝𝑟𝑖𝑐𝑒 + 𝛽2𝑛𝑒𝑤_𝑝𝑟𝑖𝑐𝑒  
- New prices = [300, 350, 400]  
- Assume unit cost of refurbished product is $150 : c = 150
- Margin: (p-c) = (refurb_price -150)

In [ ]:
refurb_data = pd.read_csv('/content/refurb_data.csv')
refurb_data.head()

,dates,new_price,refurb_price,choice
0,2014-06-01,400,264.0,refurb
1,2014-06-01,400,264.0,refurb
2,2014-06-01,400,264.0,refurb
3,2014-06-01,400,264.0,new
4,2014-06-01,400,264.0,new


In [ ]:
refurb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   dates         500 non-null    object 
 1   new_price     500 non-null    int64  
 2   refurb_price  500 non-null    float64
 3   choice        500 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 15.8+ KB


In [ ]:
refurb_demand = pd.DataFrame(refurb_data[refurb_data['choice']=='refurb'].groupby(['dates','choice'],
                                                                                  as_index=False).agg({'new_price':'mean',
                                                                                                       'refurb_price':'mean',
                                                                                                       'choice':'count'}))
refurb_demand.rename(columns={'choice':'refurb_demand'}, inplace=True)
refurb_demand

,dates,new_price,refurb_price,refurb_demand
0,2014-06-01,400.0,264.0,9
1,2014-06-02,400.0,305.0,3
2,2014-06-03,400.0,306.0,6
3,2014-06-04,400.0,283.0,7
4,2014-06-05,400.0,305.0,8
5,2014-06-06,400.0,278.0,7
6,2014-06-07,400.0,337.0,3
7,2014-06-08,400.0,286.0,8
8,2014-06-09,400.0,335.0,5
9,2014-06-10,400.0,277.0,7


In [ ]:
# Regressing refurb_demand on new and refurb prices
model_ols_2 = smf.ols(formula='refurb_demand ~ refurb_price + new_price', data=refurb_demand).fit()
print(model_ols_2.summary())

                            OLS Regression Results                            
Dep. Variable:          refurb_demand   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     7.601
Date:                Sun, 07 Apr 2024   Prob (F-statistic):            0.00241
Time:                        18:28:49   Log-Likelihood:                -60.852
No. Observations:                  30   AIC:                             127.7
Df Residuals:                      27   BIC:                             131.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.7019      3.224     -0.218   

### 2.1 Optimal refurbished price and profit | New price = 300

In [ ]:
c = 150
intercept = model_ols_2.params[0]
refurb_coef = model_ols_2.params[1]
new_coef = model_ols_2.params[2]

# Write objective: profit
# Add -1 to the minimizer if you want to maximize
# D(p)(p-c)
pi_300 = lambda r: -(intercept + refurb_coef*r + new_coef*300) * (r-c)

# Calculating optimal price p* for weekday (w=0)
optimal_refurb_300 = minimize_scalar(pi_300, method='brent')
optimal_refurb_300_price = optimal_refurb_300.x

print("When New Price = 300, Optimal Refurbished Price is: $", np.round(optimal_refurb_300_price, 2))

When New Price = 300, Optimal Refurbished Price is: $ 236.39


In [ ]:
optimal_refurb_300_profit = -optimal_refurb_300.fun

print("When New Price = $300")
print("And Optimal Refurbished Price = $", np.round(optimal_refurb_300_price, 2))
print("Corresponding profit is = $", np.round(optimal_refurb_300_profit, 2))

When New Price = $300
And Optimal Refurbished Price = $ 236.39
Corresponding profit is = $ 334.63


### 2.2 Optimal refurbished price and profit | New price = 350

In [ ]:
# Write objective: profit
# Add -1 to the minimizer if you want to maximize
# D(p)(p-c)
pi_350 = lambda r: -(intercept + refurb_coef*r + new_coef*350) * (r-c)

# Calculating optimal price p* for weekday (w=0)
optimal_refurb_350 = minimize_scalar(pi_350, method='brent')
optimal_refurb_350_price = optimal_refurb_350.x

print("When New Price = 350, Optimal Refurbished Price is: $", np.round(optimal_refurb_350_price, 2))

When New Price = 350, Optimal Refurbished Price is: $ 264.6


In [ ]:
optimal_refurb_350_profit = -optimal_refurb_350.fun

print("When New Price = $350")
print("And Optimal Refurbished Price = $", np.round(optimal_refurb_350_price, 2))
print("Corresponding profit is = $", np.round(optimal_refurb_350_profit, 2))

When New Price = $350
And Optimal Refurbished Price = $ 264.6
Corresponding profit is = $ 588.77


### 2.3 Optimal refurbished price and profit | New price = 400

In [ ]:
# Write objective: profit
# Add -1 to the minimizer if you want to maximize
# D(p)(p-c)
pi_400 = lambda r: -(intercept + refurb_coef*r + new_coef*400) * (r-c)

# Calculating optimal price p* for weekday (w=0)
optimal_refurb_400 = minimize_scalar(pi_400, method='brent')
optimal_refurb_400_price = optimal_refurb_400.x

print("When New Price = 400, Optimal Refurbished Price is: $", np.round(optimal_refurb_400_price, 2))

When New Price = 400, Optimal Refurbished Price is: $ 292.8


In [ ]:
optimal_refurb_400_profit = -optimal_refurb_400.fun

print("When New Price = $400")
print("And Optimal Refurbished Price = $", np.round(optimal_refurb_400_price, 2))
print("Corresponding profit is = $", np.round(optimal_refurb_400_profit, 2))

When New Price = $400
And Optimal Refurbished Price = $ 292.8
Corresponding profit is = $ 914.24


### 2.4 Comparison & Interpretation
Comment on the differences in the optimal refurbished price when offering the three different new prices. Next, comment on the differences in the optimal profit when offering the three different new prices.

## Part III: Pricing Analysis
---
- **Two-Segment pricing**
- **Dataset:** [none needed]

Consider the objective where you would like to know how to segment prices to obtain more potential profit. Assume that you are able to charge one price to all customers in segment 1 and another price to all customers in segment 2. Ultimately, you want to determine whether the (i) strategy of segmenting to offer two distinct prices (segmentation) outperforms the (ii) default strategy of offering a singular price (no segmentation).

Additional info:
- Linear Price-response Curve: d(p) = 10000-800p
- Cost is `$`5 : c = 5
- Also assume you will be able to price differently for customers with WTP above and below `$`7

### 3.1 Find the optimal price when only offering one price (with no segmentation)

In [ ]:
cost = 5
num_consumers = 10000
wtp_threshold = 7
max_price = 12.5

def unsegmented(p):
    return -(num_consumers - 800 * p) * (p - cost)
optimal_result = minimize_scalar(unsegmented, method='bounded', bounds=(0, max_price))
optimal_price_unsegmented = optimal_result.x
optimal_profit_unsegmented = -unsegmented(optimal_result.x)

print(f"Optimal Unsegmented Price: ${optimal_price_unsegmented}")
print(f"Corresponding Profit for Unsegmented Price: ${optimal_profit_unsegmented}")

Optimal Unsegmented Price: $8.75
Corresponding Profit for Unsegmented Price: $11250.0


### 3.2 Report optimal prices p* for segment 1 and segment 2

In [ ]:
def seg2(p):
    return -(wtp_threshold / max_price * num_consumers - 800 * p) * (p - cost)

optimal_result = minimize_scalar(unsegmented, method='bounded', bounds=(0, max_price))
optimal_price_seg1 = round(optimal_result.x, 2)
profit_seg1 = round(-unsegmented(optimal_result.x), 2)

seg2_optimal_result = minimize_scalar(seg2, method='bounded', bounds=(0, max_price))
optimal_price_seg2 = round(seg2_optimal_result.x, 2)
profit_seg2 = round(-seg2(seg2_optimal_result.x), 2)

total_profit = round(profit_seg1 + profit_seg2, 2)

print(f"Segment 1:")
print(f"Optimal Price (p*): ${optimal_price_seg1}")
print(f"Corresponding Profit: ${profit_seg1}\n")
print(f"Segment 2:")
print(f"Optimal Price (p*): ${optimal_price_seg2}")
print(f"Corresponding Profit: ${profit_seg2}\n")
print(f"Total Profit: ${total_profit}")

Segment 1:
Optimal Price (p*): $8.75
Corresponding Profit: $11250.0

Segment 2:
Optimal Price (p*): $6.0
Corresponding Profit: $800.0

Total Profit: $12050.0


### 3.3 Report the difference in profit and offer your conclusion of the two strategies (i) and (ii)

**Interpretation**

- **Single Market Strategy:**
Without segmentation, the optimal price remains at 8.75 per unit, resulting in a profit of 11,250. This approach treats the entire market as one homogeneous entity.

- **Segmented Market Approach:** Upon dividing the market into two distinct segments, varying optimal prices emerge. For segment 1, the optimal price remains unchanged at 8.75, yielding the same profit of 11,250. However, for segment 2, a lower optimal price of 6.00 is identified, generating a profit of 800.

- **Comparative Analysis:** Segmenting the market proves to be more profitable, with a total profit of 12,050, exceeding the profit generated without segmentation by 800. This underscores the efficacy of tailoring pricing strategies to specific market segments, enabling the capture of additional consumer surplus from varying price sensitivities. Such segmentation facilitates a more refined approach to pricing, optimizing profitability across diverse consumer segments.